In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from applications.common import views
from aadatapipelinecore.core.utils.spark import eject_all_caches
from aadatapipelinecore.core.pipeline.type_ import EventType, ManipulationType

sql_text='''

WITH cross_genre_raw_data_usage_penentration AS (
    SELECT COUNT(1) AS xup_raw FROM raw where metric_name = 'XUP'
);

WITH cross_genre_raw_data_genre_affinity AS (
    SELECT COUNT(1) AS xga_raw FROM raw where metric_name = 'XGA'
);

--!DEBUG:SHOW SELECT * from cross_genre_raw_data_usage_penentration;
--!DEBUG:SHOW SELECT * from cross_genre_raw_data_genre_affinity;

WITH cross_genre_unified_data_usage_penetration AS (
    SELECT count(1) AS xup_unified FROM uds_view_cross_genre WHERE date = '2019-01-31' and est_cross_genre_usage_penetration IS NOT NULL
);

WITH cross_genre_unified_data_genre_affinity AS (
    SELECT count(1) AS xga_unified FROM uds_view_cross_genre  WHERE date = '2019-01-31' and est_cross_genre_affinity IS NOT NULL
);

--!DEBUG:SHOW SELECT * from cross_genre_unified_data_usage_penetration;
--!DEBUG:SHOW SELECT * from cross_genre_unified_data_genre_affinity;

'''
class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    urn.manipulation = ManipulationType.INSERT
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    # views.init(spark, sql_text)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()

namespace = "aa.usage.genre-cross.v1"
ingest_msg = {
    "namespace": "aa.usage.genre-cross.v1",
    "job_type": "routine",
    "options": {},
    "source": [
        {
            "data_encoding": "parquet",
            "compression": "gzip",
            "name": "raw",
            "path": ["s3://aardvark-prod-dca-data/oss/GAME_CROSS_GENRE_KPI_PARQUET/version=1.0.6/range_type=MONTH/date=2019-01-31"]
        }

    ]
}

run(spark, ingest_msg, sql_text)


In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from applications.common import views
from aadatapipelinecore.core.utils.spark import eject_all_caches
from aadatapipelinecore.core.pipeline.type_ import EventType, ManipulationType

sql_text='''

WITH cross_genre_raw_data_usage_penentration AS (
    SELECT COUNT(1) AS xup_raw FROM raw where metric_name = 'XUP' and date = '2019-01-31'
);

WITH cross_genre_raw_data_genre_affinity AS (
    SELECT COUNT(1) AS xga_raw FROM raw where metric_name = 'XGA' and date = '2019-01-31'
);

--!DEBUG:SHOW SELECT * from cross_genre_raw_data_usage_penentration;
--!DEBUG:SHOW SELECT * from cross_genre_raw_data_genre_affinity;

WITH cross_genre_unified_data_usage_penetration AS (
    SELECT count(1) AS xup_unified FROM uds_view_cross_genre WHERE date = '2019-01-31' and est_cross_genre_usage_penetration IS NOT NULL
);


WITH cross_genre_unified_data_genre_affinity AS (
    SELECT count(1) AS xga_unified FROM uds_view_cross_genre  WHERE date = '2019-01-31' and est_cross_genre_affinity IS NOT NULL
);



--!DEBUG:SHOW SELECT * from cross_genre_unified_data_usage_penetration;
--!DEBUG:SHOW SELECT * from cross_genre_unified_data_genre_affinity;

'''
class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    urn.manipulation = ManipulationType.INSERT
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    # _view(spark, sql_text, None, source_data_list)
    views.init(spark, sql_text)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()

namespace = "aa.usage.genre-cross.v1"
ingest_msg = {
    "namespace": "aa.usage.genre-cross.v1",
    "job_type": "routine",
    "options": {},
    "source": [
        {
            "data_encoding": "parquet",
            "compression": "gzip",
            "name": "raw",
            "path": ["s3://aardvark-prod-dca-data/oss/GAME_CROSS_GENRE_KPI_PARQUET/version=1.0.6/range_type=MONTH/date=2019-01-31/"]
        }

    ]
}

run(spark, ingest_msg, sql_text)


In [0]:


spark.sql("SELECT * FROM uds_view_mobileweb_basic").show()


In [0]:


baisc_path = "s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/date=2018-01-04/platform=1/"
city_path = "s3://aardvark-prod-pdx-mdm-to-int/city_level/report/routine/version=1.0.0/granularity=monthly/date=2018-01-31/country_code=US/"
retention_path = ""
cross_genre_path = "s3://aardvark-prod-dca-data/oss/GAME_CROSS_GENRE_KPI_PARQUET/version=1.0.6/range_type=MONTH/date=2019-01-31/platform=1/"
domain_browser_path = "s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_BROWSER_METRICS/version=v1.0.0/granularity=MONTH/date=2019-01-31/platform=1/"
cross_genre_unified_path = "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.genre-cross.v1/fact/granularity=monthly/date=2020-06-30/"

df1 = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/usage.genre-cross.v1/fact").parquet(cross_genre_unified_path).createOrReplaceTempView("test")
spark.sql("SELECT * FROM test WHERE date = '2020-06-30' limit 10").show()